Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [40]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import logging

In [2]:
pickle_file = 'models/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [124]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    keep_prob = tf.placeholder(tf.float32)
  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = {
        'conv_w1': tf.Variable(tf.random_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)),
        'conv_w2': tf.Variable(tf.random_normal( [patch_size, patch_size, depth, depth], stddev=0.1)),
        'w3': tf.Variable(tf.random_normal( [image_size // 4 * image_size // 4 * depth, num_hidden],
                                           stddev=0.1)),
        'out': tf.Variable(tf.random_normal([num_hidden, num_labels], stddev=0.1))
    }
    
    biases = {
        'conv_b1': tf.Variable(tf.random_normal([depth], stddev=0.1)),
        'conv_b2': tf.Variable(tf.random_normal([depth], stddev=0.1)),
        'b3': tf.Variable(tf.random_normal([num_hidden], stddev=0.1)),
        'out': tf.Variable(tf.random_normal([num_labels], stddev=0.1))
    }

  # Model.
    def model(data, verbose=False):
        
        logger.info("Data: %s", data)
        conv = tf.nn.conv2d(data, weights['conv_w1'], [1, 1, 1, 1], padding='SAME')
        conv = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        logger.info("Conv1: %s", conv)
        hidden = tf.nn.relu(conv + biases['conv_b1'])
        conv = tf.nn.conv2d(hidden, weights['conv_w2'], [1, 1, 1, 1], padding='SAME')
        conv = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        logger.info('conv2: %s', conv)
        hidden = tf.nn.relu(conv + biases['conv_b2'])
        shape = hidden.get_shape().as_list()
        logger.info("shape: %s", shape)
        
        
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        logger.info("reshape: %s", reshape)
        logger.info("w3: %s", weights['w3'])
        hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, weights['w3']) +  biases['b3']), keep_prob)
        out = tf.matmul(hidden,  weights['out']) + biases['out']
        logger.info("out: %s", out)
        return out

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

INFO:root:Data: Tensor("Placeholder_1:0", shape=(64, 28, 28, 1), dtype=float32)
INFO:root:Conv1: Tensor("MaxPool:0", shape=(64, 14, 14, 16), dtype=float32)
INFO:root:conv2: Tensor("MaxPool_1:0", shape=(64, 7, 7, 16), dtype=float32)
INFO:root:shape: [64, 7, 7, 16]
INFO:root:reshape: Tensor("Reshape:0", shape=(64, 784), dtype=float32)
INFO:root:w3: <tf.Variable 'Variable_2:0' shape=(784, 64) dtype=float32_ref>
INFO:root:out: Tensor("add_3:0", shape=(64, 10), dtype=float32)
INFO:root:Data: Tensor("Const:0", shape=(10000, 28, 28, 1), dtype=float32)
INFO:root:Conv1: Tensor("MaxPool_2:0", shape=(10000, 14, 14, 16), dtype=float32)
INFO:root:conv2: Tensor("MaxPool_3:0", shape=(10000, 7, 7, 16), dtype=float32)
INFO:root:shape: [10000, 7, 7, 16]
INFO:root:reshape: Tensor("Reshape_1:0", shape=(10000, 784), dtype=float32)
INFO:root:w3: <tf.Variable 'Variable_2:0' shape=(784, 64) dtype=float32_ref>
INFO:root:out: Tensor("add_7:0", shape=(10000, 10), dtype=float32)
INFO:root:Data: Tensor("Const_1:0"

In [ ]:
num_steps = 9001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep_prob: 0.7}
        
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 2.398574
Minibatch accuracy: 18.8%


In [13]:
best 94

7.0

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---